In [1]:
!pip -qqq install langchain apify-client faiss-cpu sentence_transformers accelerate bitsandbytes
!pip -q install langchain unstructured sentence_transformers faiss-cpu huggingface_hub OpenAI
!pip -q install git+https://github.com/huggingface/transformers
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate einops
# !pip install pdfplumber

from langchain.document_loaders.base import Document
from langchain.indexes import VectorstoreIndexCreator
from langchain.utilities import ApifyWrapper
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS, Chroma
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains import RetrievalQA
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.document_loaders import ApifyDatasetLoader
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline, AutoTokenizer, AutoModelForCausalLM
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
import torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 67.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.9 MB/s eta 0:0

In [3]:
embeddings = HuggingFaceEmbeddings(model_name="intfloat/e5-large-v2")

In [4]:
!pip install pdfplumber
import pdfplumber

def pdf_loader(pdf_file):
    with pdfplumber.open(pdf_file) as pdf:
        pages = pdf.pages

        documents = []
        for page in pages:
            text = page.extract_text()
            documents.append(Document(page_content=text))

    return documents

documents = pdf_loader("football.pdf")

text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200
    )

docs = text_splitter.split_documents(documents)
db = FAISS.from_documents(docs, embeddings)


In [6]:
import torch
from instruct_pipeline import InstructionTextGenerationPipeline
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("databricks/dolly-v2-3b", padding_side="left")
model = AutoModelForCausalLM.from_pretrained("databricks/dolly-v2-3b", device_map="auto", torch_dtype=torch.bfloat16)

# generate_text = InstructionTextGenerationPipeline(model=model, tokenizer=tokenizer)


In [7]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=3000,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.2,
)

local_llm = HuggingFacePipeline(pipeline=pipe)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [18]:
qa = RetrievalQA.from_chain_type(llm=local_llm,
                                 chain_type="stuff",
                                 retriever=db.as_retriever(k=2),
                                 return_source_documents=True,
                                 verbose=True)

query = "what is football?"
result = qa(query)
result

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.




> Entering new  chain...

> Finished chain.


{'query': 'what is football?',
 'result': " Football is a fast-moving game played between two teams of eleven players each. Each player has a ball and must pass it to another player on his team without letting go of his own handball. Players run onto a pitch marked out with lines representing the boundaries of the field. A referee officiates the game while referees for youth levels are usually coaches. There are four quarters followed by a half-time break where substitutions are allowed but not during actual play. The objective of the team that gets the ball through their opponents' defense three times in a row is scored. The first person to reach the opposition's goal line with the ball wins possession. The object of the game is to move the ball across the field until your teammate(s) score a goal. It is considered good form to support the strikers when they have the ball. This will help them create chances to get the ball into the net.\n\nThere are many variations of this basic rule 

In [19]:
result['result']

" Football is a fast-moving game played between two teams of eleven players each. Each player has a ball and must pass it to another player on his team without letting go of his own handball. Players run onto a pitch marked out with lines representing the boundaries of the field. A referee officiates the game while referees for youth levels are usually coaches. There are four quarters followed by a half-time break where substitutions are allowed but not during actual play. The objective of the team that gets the ball through their opponents' defense three times in a row is scored. The first person to reach the opposition's goal line with the ball wins possession. The object of the game is to move the ball across the field until your teammate(s) score a goal. It is considered good form to support the strikers when they have the ball. This will help them create chances to get the ball into the net.\n\nThere are many variations of this basic rule set depending on the level of competition.

In [20]:
#default prompt
qa.combine_documents_chain.llm_chain.prompt.template

"Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:"